In [43]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data
import importlib
import life.lib
import life.lib as lib
importlib.reload(lib)
LOG = lib.simple_log.LOG
import math
from matplotlib import pyplot as plt
%matplotlib
plt.ion()
lib.utils.set_seed()
print(torch.__version__)
np.set_printoptions(precision=3)
%precision 4



Using matplotlib backend: TkAgg
2.0.0+cu117


'%.4f'

In [5]:
if "train_x" not in globals():
    x, y = lib.datasets.get_mnist_train()
    train_y = y % 2
    train_x = torch.reshape(x, (len(x), -1))
    x, y = lib.datasets.get_mnist_test()
    test_y = y % 2
    test_x = torch.reshape(x, (len(x), -1))



In [46]:
# network = lib.nets.PYRAMID()
#network = lib.nets.create_livenet_odd_2()
# train_x, train_y = lib.datasets.get_odd()
network = lib.livenet.LiveNet(train_x.shape[1], None, torch.max(train_y) + 1)
network = lib.nets.PERCEPTRON(train_x.shape[1], torch.max(train_y) + 1)
criterion = lib.nets.criterion_n

def create_optimizer(net: nn.Module):
    if net.__class__.__name__ == "LiveNet":
        print("LiveNet")
        net: lib.livenet.LiveNet
        optimizer = lib.livenet.LiveNetOptimizer(net, decay=0.0)
        # optimizer = torch.optim.Adam(net.parameters())
    else:
        print("Torch")
        optimizer = torch.optim.Adam(net.parameters(), betas=(0.9, 0.999))
    return optimizer

optimizer = create_optimizer(network)


Torch


In [47]:
params = [p for p in network.named_parameters()]
logits = network.forward(train_x)
for name, p in params:
    print(f"{name} {p.detach().numpy()}")
pred = network.forward(train_x)
loss = criterion(pred, train_y)
print(loss)
pred.detach().numpy()
# torch.onnx.export(network, train_x, "/home/spometun/model.onnx", verbose=False)




linear1.weight [[-0.     0.019 -0.029 ...  0.022  0.004  0.002]
 [-0.02  -0.015 -0.01  ... -0.02  -0.006 -0.03 ]]
linear1.bias [-0.02   0.015]
tensor(1.0939, grad_fn=<DivBackward0>)


array([[ 0.273,  0.635],
       [-0.329,  0.542],
       [ 0.153, -0.004],
       ...,
       [ 0.304,  0.578],
       [ 0.077,  0.333],
       [-0.019,  0.095]], dtype=float32)

In [48]:
# network.context.decay = 0.00
params_history = []


In [49]:
optimizer.param_groups[0]["lr"] = 0.00001
g = optimizer.param_groups
batch_size = 1000

n_epochs = 80
for i in range(n_epochs):
    params = [(p[0], p[1].detach().clone().numpy()) for p in network.named_parameters()]
    params_history.append(params)
    for start, end in lib.gen_utils.index_batcher(batch_size, len(train_x)):
        x = train_x[start:end]
        y = train_y[start:end]
        pred = network.forward(x)
        loss = criterion(pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if i % 1 == 0:
        LOG(f"{loss.detach().numpy():.3f}")



Iˈ0.000 1.058 
Iˈ0.063 1.015 
Iˈ0.120 0.976 
Iˈ0.177 0.940 
Iˈ0.233 0.907 
Iˈ0.295 0.877 
Iˈ0.353 0.849 
Iˈ0.412 0.824 
Iˈ0.469 0.800 
Iˈ0.529 0.778 
Iˈ0.588 0.757 
Iˈ0.643 0.738 
Iˈ0.699 0.721 
Iˈ0.758 0.704 
Iˈ0.814 0.689 
Iˈ0.873 0.674 
Iˈ0.931 0.661 
Iˈ0.991 0.648 
Iˈ1.046 0.636 
Iˈ1.101 0.625 
Iˈ1.156 0.614 
Iˈ1.219 0.604 
Iˈ1.277 0.594 
Iˈ1.334 0.585 
Iˈ1.391 0.576 
Iˈ1.449 0.568 
Iˈ1.505 0.560 
Iˈ1.560 0.553 
Iˈ1.614 0.546 
Iˈ1.674 0.539 
Iˈ1.731 0.532 
Iˈ1.787 0.526 
Iˈ1.844 0.520 
Iˈ1.905 0.514 
Iˈ1.962 0.509 
Iˈ2.020 0.503 
Iˈ2.076 0.498 
Iˈ2.132 0.493 
Iˈ2.190 0.488 
Iˈ2.246 0.484 
Iˈ2.311 0.479 
Iˈ2.372 0.475 
Iˈ2.428 0.471 
Iˈ2.484 0.467 
Iˈ2.560 0.463 
Iˈ2.617 0.459 
Iˈ2.677 0.455 
Iˈ2.748 0.452 
Iˈ2.811 0.448 
Iˈ2.874 0.445 
Iˈ2.936 0.442 
Iˈ2.997 0.439 
Iˈ3.054 0.435 
Iˈ3.111 0.432 
Iˈ3.165 0.430 
Iˈ3.220 0.427 
Iˈ3.273 0.424 
Iˈ3.330 0.421 
Iˈ3.386 0.419 
Iˈ3.443 0.416 
Iˈ3.498 0.414 
Iˈ3.558 0.411 
Iˈ3.616 0.409 
Iˈ3.672 0.407 
Iˈ3.728 0.404 
Iˈ3.785 0.402 
Iˈ3.841 0.

In [10]:
pred = network(train_x)
pred_bin = np.argmax(pred.detach().numpy(), axis=1, keepdims=True)
diff = train_y - pred_bin
len(diff[diff != 0]) / len(diff)

0.1416

In [50]:
def param_picker(param):
    return param[0][1][0][382].item()
    # return np.max(np.abs(param[0].numpy()))

def get_param_values(history, picker):
    values = []
    for entry in history:
        values.append(picker(entry))
    return values

values = get_param_values(params_history, param_picker)
# plt.figure(figsize=(12, 9))
# plt.plot(values)
# plt.grid()

accum = lib.stat_utils.AccumStat()
accum.add_value(params_history[0])
accum.plot()




In [ ]:
import matplotlib.pyplot as plt
plt.plot(all_losses)
plt.ylabel('Loss')
plt.show()

In [ ]:
# list(network.parameters())[0].data = torch.Tensor([[10.], [-10]])
# list(network.parameters())[1].data = torch.Tensor([-5., 15])
np.set_printoptions(precision=2)
params = list(network.parameters())
par = []
for p in params:
    par.append(p.detach())
    print (par[-1])

In [ ]:
o1 = train_x @ par[1].T + par[2]
o1

In [ ]:
a1 = torch.sigmoid(o1)
a1

In [ ]:
o2 = a1[:, 0] + a1[:, 1] - 1
mseloss(o2, train_y)

In [ ]:
o2 @ par[2].T

In [ ]:
network.forward(train_x)

In [ ]:
1345639 * 5145

In [ ]:
import pickle
import numpy as np

class A:
    def __init__(self):
        self.v = 1
    def __del__(self):
        print(f"del {self.v}")

a = A()
b = A()
b.v = 2
l = [b]
print("start")
del a
print("after del")
l[0].v
a.v
